# Montagem do módelo

## Importando bibliotecas

Primeiro importe as bibliotecas necessárias para a montagem do modelo.

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

## Carregue os dados

O TensorFlow facilita a importação do MNIST, então apenas importamos de `datasets` e dividimos em conjuntos de treinamento e teste.

In [3]:
# Carregamento dos dados de treino e teste
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalizando os valores dos pixels para o intervalo 0-1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Redimensionando as imagens
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

## Construção do modelo

Aqui definimos um modelo com as seguintes configurações:
- **Camada Convolucional (Conv2D)**: para detectar as características das imagens.
  - `32 filtros`: Cada filtro detecta características diferentes.
  - `Tamanho do filtro (3x3)`: Área da imagem que cada filtro examina.
  - `Função de Ativação 'ReLU'`: Introduz não-linearidade no modelo, transformando valores negativos em zero. 

- **Camada de Pooling**: Reduz a dimensionalidade espacial.
  - `Pooling Máximo (2x2)`: Reduz a dimensão espacial pegando o valor máximo de um quadrado 2x2.

- **Flatten**: Converte mapas de características 3D em um vetor 1D.

- **Camadas Densas**: Realizam a classificação com base nas características detectadas.
  - `64 neurônios`: Número de neurônios na camada oculta.
  - `10 neurônios na camada de saída`: Corresponde aos 10 dígitos do MNIST.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Adicionando camadas densas no final
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

## Compilação do modelo

Agora que o modelo está definido, precisamos configurar o processo de treinamento. Para isso primeiro compilamos o modelo com as seguintes configurações:

- **Optimizer 'Adam'**: Ajusta os pesos da rede para minimizar o erro.
- **Loss 'Sparse Categorical Crossentropy'**: Compara as previsões do modelo com os rótulos verdadeiros.
- **Métrica 'Accuracy'**: Avalia a porcentagem de previsões corretas.

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Treinamento do modelo

Agora treinamos o modelo com os dados de treinamento com 3 épocas e validamos com os dados de teste. 

In [6]:
model.fit(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))

Epoch 1/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1461 - accuracy: 0.9558 - val_loss: 0.0508 - val_accuracy: 0.9840
Epoch 2/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0466 - accuracy: 0.9856 - val_loss: 0.0314 - val_accuracy: 0.9896
Epoch 3/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0330 - accuracy: 0.9898 - val_loss: 0.0291 - val_accuracy: 0.9922


## Avaliação do modelo

Por fim, avaliamos o modelo com os dados de teste e imprimimos a acurácia.

In [7]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.0291 - accuracy: 0.9922 - 917ms/epoch - 3ms/step

Test accuracy: 0.9922000169754028


# Teste do modelo

## Script

Agora que o modelo está treinado, podemos testá-lo com uma imagem de um dígito escrito à mão. Para isso, primeiro importamos a imagem e a convertemos para um array de 28x28. Depois, usamos o modelo para prever o dígito e imprimimos o resultado.

In [1]:
import cv2

In [8]:
# Caminho da imagem
imagem_caminho = './numero_a_mao.jpg'

In [9]:
# Carregamento e formatação da imagem
img = cv2.imread(imagem_caminho, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))

img = img / 255.0

img = np.expand_dims(img, axis=[0, -1])

In [22]:
# Prevendo a imagem
previsao = model.predict(img)

previsao_label = np.argmax(previsao, axis=1)

print(f"Número previsto: {previsao_label[0]}")

1/1 [==============================] - 0s 15ms/step


'Número previsto: 8'